In [1]:
from pandas import DataFrame
import pandas as pd

In [2]:
# create the frame
frame = pd.read_csv("fortune500.csv")

In [3]:
# get the industry list
industries = set(list(frame.ix[:, 'industry']))

In [23]:
# decompile the parts of the dataframe that we want into a [company, industry] list for d3 to use
dat = frame.ix[:, ['company', 'industry']].to_json(orient='values')

In [17]:
# hand off the dataframe using IPython
from IPython.display import Javascript
Javascript("""
           window.dat={};
           """.format(dat))

<IPython.core.display.Javascript object>

In [28]:
%%javascript

require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3'
    }
});

<IPython.core.display.Javascript object>

In [30]:
%%javascript

console.log("Hello!")
console.log(dat)

<IPython.core.display.Javascript object>

In [34]:
%%javascript
require(['d3'], function(d3){
//     /*
//         Jupyter notebooks save their state between runtimes
//         so the chart needs to be explicitly destroyed and
//         recreated every time the code is run.
//     */
//     $("#chart").remove();
//     element.append("<div id='chart'></div>");

    
//     var dataset = [ 5, 10, 15, 20, 25 ]; 
    console.log("dat")
});

<IPython.core.display.Javascript object>

In [26]:
%%javascript
console.log("hello world!")

<IPython.core.display.Javascript object>